In [1]:
from __future__ import print_function

import os
import torch
from PIL import Image

import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import torchvision.transforms as transforms
import torchvision.models as models
import torch.utils.data
import torchvision.datasets

from torch.autograd import Variable

import copy    
import time
import numpy as np

Read Images

In [2]:

assert(os.getcwd()=="/Users/wangyiming/Documents/ncsu course/ncsu 2019 spring/ECE/Project 2")
resolution = 60
Train_root = "resolution"+str(resolution)+"by"+str(resolution)+"/extracted_pics/Train/"
Test_root = "resolution"+str(resolution)+"by"+str(resolution)+"/extracted_pics/Test/"
Train = os.listdir(Train_root)
Test = os.listdir(Test_root)


Define transformation and image flow

In [3]:
#without normalization
loader1=transforms.ToTensor()# if not normalize then in range[0,1]

#normalization for simple feedforward neural network
loader2=transforms.Compose(
    [transforms.ToTensor(),#convert an image to tensor
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#normalization for LeNet5
loader3=transforms.Compose(
        [transforms.Resize((32,32)),
         transforms.ToTensor(),
         transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

def load_images_flow(batch_size,root,which_trans):
    if(which_trans == 1):
         transform = loader1
    if(which_trans == 2):
         transform = loader2
    if(which_trans == 3):
         transform = loader3
    
    train_set = torchvision.datasets.ImageFolder(root=root, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
    
    return train_loader

Define loss function and optimizer

In [4]:
#??how to tune learning rate and weight decay
def createLossAndOptimizer_1d(net, learning_rate = 0.001, weight_decay = 0):#weight_decay: tuning parameter of L2 term
     #Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=weight_decay) 
    
    return(loss, optimizer)

In [5]:

def train_net(net, which_model, whether_norm, batch_size, n_epochs, learning_rate, weight_decay):
    assert(which_model == "NN" or which_model == "LeNet5")# the input should be reasonable
    
    #choose right transformation
    if(whether_norm == True):
        which_trans = 1
    elif(which_model == "NN"):
        which_trans = 2
    else:
        which_trans = 3
        
    #Get training data and test data
    train_loader = load_images_flow(batch_size=batch_size,
                                    root=Train_root,
                                    which_trans=which_trans)
    test_loader = load_images_flow(batch_size=batch_size,
                                   root=Test_root,
                                   which_trans=which_trans)
     
    
    n_batches = len(train_loader)
    loss, optimizer =  createLossAndOptimizer_1d(net, learning_rate = learning_rate, weight_decay = weight_decay )
    
    #Time for printing
    start_time = time.time()
    print_every = n_batches // 10
        
    for epoch in range(n_epochs):
        running_loss = 0
        total_train_loss = 0
        
        for i, data in enumerate(train_loader):
            (inputs,labels) = data
            
            if(which_model == "NN"):
                 inputs = inputs.view(inputs.size()[0],3*60*60)#flatten
                 #inputs = inputs.view(batch_size,3*60*60) not batch size since 
            
            inputs, labels = Variable(inputs), Variable(labels)
            
            optimizer.zero_grad() # whether zero setting is okay ?
         
            #Forward pass, backward pass, optimize
            outputs = net(inputs) #why ? same as forward
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()       #??? wh
            
            #print statistics
            running_loss += loss_size.data
            total_train_loss += loss_size.data
            
            #print every 10th batch
            if (i+1) % (print_every+1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100*(i+1)/n_batches), running_loss/print_every, time.time()-start_time))
                #reset running loss and time
                running_loss = 0.0
                start_time = time.time()
                
        total_test_loss = 0
        for inputs, labels in test_loader:  
            if (which_model == True):
                inputs = inputs.view(inputs.size()[0], 3*60*60)
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Forward pass
            test_outputs = net(inputs)
            test_loss_size=loss(test_outputs, labels)
            total_test_loss += test_loss_size.data
        
        print("Test loss = {:.2f}".format(total_test_loss / len(test_loader)))
        
    print("Training finished, took {:.2f}s".format(time.time() - start_time))
    
            

Simple Feedforward Neural Network

In [6]:

class Two_Layers_NN(torch.nn.Module):
    
    def __init__(self):
        super(Two_Layers_NN,self).__init__()
        input_size = 3*60*60; hidden_size = 1000; output_size = 2
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
        return out
    

LeNet5 Neural Network

In [7]:
class LeNet5(torch.nn.Module):
    def __init__(self):
        super(LeNet5,self).__init__()
        
        self.conv1 = torch.nn.Conv2d(3, 6, kernel_size=(5,5), stride=1)#input 3 channels, output 6 channels
        
        self.maxpool = nn.MaxPool2d(kernel_size=(2,2),stride=2)
        # torch.nn.AdaptiveAvgPool2d() can avoid overfitting
        self.conv2 = torch.nn.Conv2d(6, 16, kernel_size=(5,5), stride=1)#output may not be the times of input
        
        self.fc1 = torch.nn.Linear(5*5*16,120)
        
        self.fc2 = torch.nn.Linear(120,84)
        
        self.fc3 = torch.nn.Linear(84,2)
        
    def forward(self,x):
        x = self.conv1(x) #input 32*32*3 output 28*28*6
        
        #x = self.batchnorm1(x)
        
        x = self.maxpool(x) #output 14*14*6
        
        x = self.conv2(x) #output 10*10*16
        
        
        x = self.maxpool(x) #output 5*5*16
        
        x = x.view(-1, 5 * 5 * 16)#flatten
        
        x = self.fc1(x)
        
        x = self.fc2(x)
        
        x = self.fc3(x)
        return(x)

Train Simple Feedforward Neural Network

In [8]:
NN = Two_Layers_NN()
train_net(NN, which_model = "NN", whether_norm = True, batch_size=10, n_epochs=5, 
          learning_rate=0.001, weight_decay=0.0001)
   

Epoch 1, 10% 	 train_loss: 1.94 took: 21.46s
Epoch 1, 20% 	 train_loss: 0.38 took: 19.56s
Epoch 1, 30% 	 train_loss: 0.36 took: 20.68s
Epoch 1, 40% 	 train_loss: 0.42 took: 21.15s
Epoch 1, 50% 	 train_loss: 0.39 took: 19.65s
Epoch 1, 60% 	 train_loss: 0.53 took: 22.74s
Epoch 1, 70% 	 train_loss: 0.39 took: 22.71s
Epoch 1, 80% 	 train_loss: 0.47 took: 22.03s
Epoch 1, 90% 	 train_loss: 0.40 took: 22.43s


RuntimeError: size mismatch, m1: [1800 x 60], m2: [10800 x 1000] at /Users/soumith/mc3build/conda-bld/pytorch_1549597882250/work/aten/src/TH/generic/THTensorMath.cpp:940

Train LeNet 5

In [ ]:
LeNet5 = LeNet5()
train_net(LeNet5, which_model = "LeNet5", whether_norm = True, batch_size=10, n_epochs=5, 
          learning_rate=0.001, weight_decay=0.0001)
                       